In [99]:
import random
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import statistics
import scipy.stats as stats
import pygame
import time

## Doctor and Patient classes

In [100]:
# Constants
WIDTH, HEIGHT = 800, 600
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0,0,255)

# Define Patient class
class Patient:
    def __init__(self, NO):
        self.priority = 0
        self.NO = NO
        self.arrival = 0.0
        self.service_time1 = 0.0
        self.service_time2 = 0.0
        self.end1 = 0.0
        self.end2 = 0.0
        self.wt1 = 0.0
        self.wt2 = 0.0
        self.twt = 0.0
        self.position = [50, 50 + 50 * NO]
        self.color = RED if self.priority == 3 else BLUE

    def __eq__(self, other):
        return self.NO == other.NO

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, self.position, 18)
        patient_no_text = font.render(str(self.NO), True, BLACK)
        screen.blit(patient_no_text, (self.position[0] - 5, self.position[1] - 10))
    
    def priorityy(self):
        global gen_i
        if lst[gen_i] > 0.85:
            self.priority = 3
        else:
            self.priority = 1
        gen_i += 1

# Define Doctor class
class Doctor:
    def __init__(self, number, status=0):
        self.number = number
        self.status = status # 0 means idle and 1 means busy
        self.svr = 0.0
        self.num_visited = 0
        self.total_q = 0
        self.total_time_q = 0.0
        self.position = [300 if number == 1 else 500, 200]
        self.queue = []

    def __eq__(self, other):
        return self.number == other.number

    def draw(self, screen):
        pygame.draw.rect(screen, BLACK, (*self.position, 60, 40))
        text = font.render(f'Dr. {self.number}', True, WHITE)
        screen.blit(text, (self.position[0] + 5, self.position[1] + 5))
        for i, patient in enumerate(self.queue):
            patient.position = [self.position[0] - (i + 1) * 50, self.position[1] + 50]
            patient.draw(screen)

In [101]:
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

## Random number generator

In [102]:
class CLCG:
    def __init__(self, K):
        self.K = K
        self.A = [0.0 for i in range(K)]
        self.m  = [0.0 for i in range(K)]
        self.length = 0
        self.seed = [1 for i in range(K)]
        self.c = [0.0 for i in range(K)]

    def element_wise_multiply(self, vector1, vector2):
        result = [vector1[i] * vector2[i] for i in range(len(vector1))]
        return result
    
    def element_wise_remainder(self, vector1, vector2):
        result = [vector1[i] % vector2[i] for i in range(len(vector1))]
        return result
    
    def next(self,x = []):
        result = 0
        for i in range(len(x)):
            result += (((-1)**i) * x[i])
        result = result % (self.m[0] - 1)
        if result ==0:
            result = (self.m[0] - 1) / self.m[0]
        else:
            result = result / self.m[0]
        return result

    def maxlength(self):
        P = 1
        for i in range(self.K):
            P = P * (self.m[i] - 1)
        P = P / pow(2, self.K - 1)
        return P
    def generate(self, length):
        X = np.empty([length, self.K])
        X[0] = self.seed
        result = np.empty(length)
        for i in range(1, length):
            
            X[i] = self.element_wise_remainder([x + y for x, y in zip(self.element_wise_multiply(X[i-1], self.A), self.c)], self.m) 

        for i in range(length):
            result[i] = self.next(X[i])
            
        return result
    def d_alpha(self, a, n):
        result = math.sqrt(((-1) / (2 * n)) * math.log(2 * a))
        return result
    
    def KS_test(self, data : np.array, a = 0.05):
        n = len(data)
        sorted_data = np.sort(data)
        ecdf = np.arange(1, n + 1) / n  # Empirical CDF
        cdf = sorted_data  # CDF of the uniform distribution in the interval [0, 1]
        d_stat = np.max(np.abs(ecdf - cdf))
        
        # Calculate critical value
        d_alpha = self.d_alpha(a , n)
        if d_alpha >= d_stat :
            print("KS Test: Fail to reject null hypothesis (data follows uniform distribution)")
            return True
        else:
            print("KS Test: Reject null hypothesis (data does not follow uniform distribution)")
            return False
    

In [103]:
def st(lst, mu, var): # calculates the service time of a customer
    global gen_i
    R = lst[gen_i]
    R2 = lst[gen_i + 1]
    R3 = lst[gen_i + 2]
    Z1 = math.sqrt((-2) * math.log(R)) * math.cos(2 * math.pi * R2)
    X1 = mu + Z1 * (var)
    Z2 = math.sqrt((-2) * math.log(R)) * math.sin(2 * math.pi * R2)
    X2 = mu + Z2 * (var)
    if R3 > 0.5:
        result = X1
    else:
        result = X2
    gen_i += 3
    return abs(result)

def at(Tnow, lst, mu, var): # calculates the arrival time of a customer
    global gen_i
    R = lst[gen_i]
    R2 = lst[gen_i + 1]
    R3 = lst[gen_i + 2]
    Z1 = math.sqrt((-2) * math.log(R)) * math.cos(2 * math.pi * R2)
    X1 = mu + Z1 * (var)
    Z2 = math.sqrt((-2) * math.log(R)) * math.sin(2 * math.pi * R2)
    X2 = mu + Z2 * (var)
    if R3 > 0.5:
        result = X1
    else:
        result = X2
    
    gen_i += 3
    
    return abs(result) + Tnow


In [104]:
ty = CLCG(2)
ty.A = [48271, 40692]
ty.m = [60013, 60017]
ty.seed = [1235, 5879]
maximum_length = ty.maxlength()
lst = ty.generate(100000)

## arrival

In [105]:
#arrival
def arrival(i,Patients):
    global FEL, Tnow, Doctors, gen_i, twt1, twt2, twt3, Q1,Q2,Q3
    c = Patient(i + 1)
    c.arrival = at(Tnow, lst,20,4)
    if c.arrival < T:
        FEL.append([0, c.arrival, c])
        c.priorityy()
        c.color = RED if c.priority == 3 else BLUE
        Patients.append(c)
    if Patients[i].priority ==1:
        if Doctors[0].status == 0:
            Doctors[0].status = 1
            Patients[i].service_time1 = st(lst, 15, 10)
            if Tnow > Ti:
                Doctors[0].num_visited += 1
                Doctors[0].svr += Patients[i].service_time1
            Patients[i].end1 = Patients[i].service_time1 + Tnow
            
            FEL.append([1, Patients[i].end1, Patients[i]])
            
        else:
            Q1.append(Patients[i])

    elif Patients[i].priority ==3:
        if Doctors[0].status == 0:
            Doctors[0].status = 1
            Patients[i].service_time1 = st(lst, 30, 5)
            if Tnow > Ti:
                Doctors[0].num_visited += 1
                Doctors[0].svr += Patients[i].service_time1
            Patients[i].end1 = Patients[i].service_time1 + Tnow
            
            FEL.append([1, Patients[i].end1, Patients[i]])
        else:
            Q3.append(Patients[i])



## Departures

In [106]:
def departure1(patient):
    global twt1, twt2, twt3, FEL, Tnow, Patients, Doctors, gen_i, Q1,Q2,Q3, Ti
    if not Q3:
        if not Q1:
            Doctors[0].status = 0
        else:
            new_patient = Q1.pop(0)
            new_patient.service_time1 = st(lst, 15, 10)
            new_patient.end1 = new_patient.service_time1 + Tnow
            new_patient.wt1 = new_patient.end1 - new_patient.service_time1 - new_patient.arrival
            if Tnow > Ti:
                twt1 += new_patient.wt1
                Doctors[0].num_visited += 1
                Doctors[0].svr += new_patient.service_time1
            FEL.append([1, new_patient.end1, new_patient])
    else:
        new_patient = Q3.pop(0)
        new_patient.service_time1 = st(lst, 30, 5)
        new_patient.end1 = new_patient.service_time1 + Tnow
        new_patient.wt1 = new_patient.end1 - new_patient.service_time1 - new_patient.arrival
        if Tnow > Ti:
            twt3 += new_patient.wt1
            Doctors[0].num_visited += 1
            Doctors[0].svr += new_patient.service_time1
        FEL.append([1, new_patient.end1, new_patient])

    if Doctors[1].status == 0:
        Doctors[1].status = 1

        if patient.priority == 1:
            #patient.priority = 2
            
            patient.service_time2 = st(lst,10, 7)
            
            patient.end2 = patient.service_time2 + Tnow
            patient.wt2 = patient.end2 - patient.service_time2 - patient.end1
            if Tnow > Ti:
                twt2 += patient.wt2
                Doctors[1].svr += patient.service_time1
                Doctors[1].num_visited += 1
            FEL.append([2, patient.end2, patient])

        elif patient.priority == 3:
            #patient.priority = 2
            patient.service_time2 = st(lst,28, 13)
            patient.end2 = patient.service_time2 + Tnow
            patient.wt2 = patient.end2 - patient.service_time2 - patient.end1
            if Tnow > Ti:
                twt2 += patient.wt2
                Doctors[1].svr += patient.service_time1
                Doctors[1].num_visited += 1
            FEL.append([2, patient.end2, patient])
    else:
        Q2.append(patient)


In [107]:
def departure2(patient):
    global twt1, twt2, twt3, FEL, Tnow, Patients, Doctors, gen_i, Q1,Q2,Q3
    if not Q2:
        Doctors[1].status = 0
    else:
        new_patient = Q2.pop(0)

        if new_patient.priority == 1:
            #new_patient.priority = 2
            new_patient.service_time2 = st(lst,10, 7)
            new_patient.end2 = new_patient.service_time2 + Tnow
            new_patient.wt2 = new_patient.end2 - new_patient.service_time2 - new_patient.end1
            if Tnow > Ti:
                twt2 += new_patient.wt2
                Doctors[1].num_visited += 1
                Doctors[1].svr += new_patient.service_time2
            FEL.append([2, new_patient.end2, new_patient])

        elif new_patient.priority == 3:
            #new_patient.priority = 2
            new_patient.service_time2 = st(lst,28, 13)
            new_patient.end2 = new_patient.service_time2 + Tnow
            patient.wt2 = patient.end2 - patient.service_time2 - patient.end1
            if Tnow > Ti:
                twt2 += new_patient.wt2
                Doctors[1].num_visited += 1
                Doctors[1].svr += new_patient.service_time2
            FEL.append([2, new_patient.end2, new_patient])
            


## Initialization

In [108]:
Doctors = [Doctor(i) for i in range(1,3)]
initialization_patients =[]
Patients = [0]
Q1,Q2,Q3 = [] , [], []
Q1_length, Q2_length, Q3_length = [], [] ,[]
Q1i_length, Q2i_length, Q3i_length = [], [] ,[]
Tnow_list = []
twt1, twt2, twt3 = 0.0, 0.0, 0.0
FEL =[]
T = 32 * 24 * 60
Ti = 2 * 24 *60
gen_i = 0
first_patient = Patient(0)
first_patient.arrival = 0
first_patient.priorityy()
initialization_patients.append(first_patient)
FEL.append([0, 0, first_patient])

## Simulation Controller

In [109]:

i = 0 #for initialization(first 2 days)
j = 0 #for main simulation (30 days)
while True:
    FEL = sorted(FEL, key=lambda x: x[1])  # Sort based on time
    if FEL:
        temp = FEL.pop(0)
        Tnow = temp[1]
        if Tnow > Ti:
            Q1_length.append(len(Q1))
            Q2_length.append(len(Q2))
            Q3_length.append(len(Q3))
            Tnow_list.append(Tnow)
        else:
            Q1i_length.append(len(Q1))
            Q2i_length.append(len(Q2))
            Q3i_length.append(len(Q3))
        if T > Tnow:
            
            if temp[0] == 0:
                if Tnow < Ti:
                    arrival(i, initialization_patients)
                    Patients[0] = initialization_patients[-1]
                    i = i + 1
                else:
                    arrival(j, Patients)
                    j += 1
            elif temp[0] == 1:
                #departure    
                departure1(temp[2])
            
            elif temp[0] == 2:
                departure2(temp[2])
        else:
            break
    else:
        break


## calculations

## Result

In [110]:
initialization_patients.pop()
Patients[0].NO = 0

In [111]:
final_table = []
header = ['patient No',
           'clock of arrival',
             'time service doctor 1 begins',
               'service time doctor 1',
                 'time service doctor 1 end',
                 'time service doctor 2 begins',
                   'service time doctor 2',
                     'time service doctor 2 end',
                       'priority',
                         'total waiting time',
                           'waiting time behind doctor1',
                             'waiting time behind doctor2',
                              'color'
          ]
main_patients = sorted(Patients, key=lambda x: x.NO)

for patient in main_patients:
    final_table.append([patient.NO + 1, patient.arrival, patient.end1 - patient.service_time1, patient.service_time1, patient.end1, patient.end2 - patient.service_time2, patient.service_time2, patient.end2,patient.priority, patient.twt, patient.wt1, patient.wt2, patient.color])

In [112]:
df = pd.DataFrame(final_table)
df.columns = header
#columns_to_convert = ['clock of arrival', 'time service 1 begins','time service end 1','time service 2 begins', 'time service end 2']
#for col in columns_to_convert:
#    df[col] = df[col].apply(convert_minutes_to_dd_hh_mm_ss)
#df.tail(50)

In [113]:
Patients[0]

In [114]:
patients = Patients.copy()

doctor1 = Doctors[0]
doctor2 = Doctors[1]

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Hospital Simulation')
clock = pygame.time.Clock()

# Font for displaying text
font = pygame.font.Font(None, 36)




# Queues for doctors
priority_queue = []
non_priority_queue = []
# Animation function
def animate(sim_speed=1, start_time_offset=0):
    running = True
    elapsed_time = start_time_offset

    while running:
        screen.fill(WHITE)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Increment elapsed time
        elapsed_time += sim_speed / 60.0  # sim_speed units per frame (assuming 60 FPS)

        # Draw simulation time
        sim_time_text = font.render(f'Time: {elapsed_time:.2f}', True, BLACK)
        screen.blit(sim_time_text, (10, 10))

        # Draw doctors
        doctor1.draw(screen)
        doctor2.draw(screen)

        # Manage queues and draw patients based on elapsed time
        for patient in patients[:]:
            if elapsed_time >= patient.arrival and elapsed_time < patient.end2:
                if elapsed_time < patient.end1:
                    if patient.priority == 3 and patient not in priority_queue:
                        priority_queue.append(patient)
                    elif patient.priority == 1 and patient not in non_priority_queue:
                        non_priority_queue.append(patient)
                elif elapsed_time >= patient.end1 and elapsed_time < patient.end2:
                    if patient in priority_queue:
                        priority_queue.remove(patient)
                    if patient in non_priority_queue:
                        non_priority_queue.remove(patient)
                    if patient not in doctor2.queue:
                        doctor2.queue.append(patient)
            elif elapsed_time >= patient.end2:
                if patient in doctor2.queue:
                    doctor2.queue.remove(patient)
                patients.remove(patient)
        
        # Draw priority queue
        for i, patient in enumerate(priority_queue):
            patient.position = [250 - (i * 30), 200]
            patient.draw(screen)

        # Draw non-priority queue
        for i, patient in enumerate(non_priority_queue):
            patient.position = [250 - (i * 30), 250]
            patient.draw(screen)

        # Draw doctor 2 queue
        unique_patients = []
        for patient in doctor2.queue:
            if patient not in unique_patients:
                unique_patients.append(patient)

        for i, patient in enumerate(unique_patients):
            patient.position = [450 - (i * 30), 250]
            patient.draw(screen)

        pygame.display.flip()
        clock.tick(60)  # Assuming 60 FPS

    pygame.quit()

# Run the animation with adjustable start time
start_time_offset = 2 * 24 * 60
animate(sim_speed=5, start_time_offset=start_time_offset)